<a href="https://colab.research.google.com/github/Koffi26/Hello/blob/master/Rev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NLP Using Sephora Comments


---




## Files directions and modules


In [ ]:
inputPath = "/content/drive/MyDrive/Reviews/raw"
outputPath = "/content/drive/MyDrive/Reviews/cleaned"

In [178]:
import pandas as pd
import re
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

## Get Cleaned Informations

### Get General Information


In [ ]:
dbs = pd.read_json(f"{inputPath}/productInformations.json")
dbs['skuId'] = dbs['skuId'].astype(str)
information = ["skuId","brandName","displayName","isNew","listPrice","productName","size","productId",
'Category1', 'Category2', 'Category3', 'brand','lovesCount', 'rating', 'reviews',
  'update']
GeneralInfo = dbs[information]
GeneralInfo['listPrice'] = GeneralInfo['listPrice'].fillna(0)
GeneralInfo['listPrice'] = GeneralInfo['listPrice'].apply(lambda x : float(str(x).replace("$","")))
GeneralInfo.drop_duplicates(subset="skuId",inplace=True)
GeneralInfo.to_csv(f"{outputPath}/AgregateInfo.csv",index=False)

<ipython-input-3-3b5ec7a42256>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GeneralInfo['listPrice'] = GeneralInfo['listPrice'].fillna(0)
<ipython-input-3-3b5ec7a42256>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  GeneralInfo['listPrice'] = GeneralInfo['listPrice'].apply(lambda x : float(str(x).replace("$","")))
<ipython-input-3-3b5ec7a42256>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/

### Get Highlights

In [ ]:
highlight_df = []
for idx,i in enumerate(dbs['highlights']) :
  dt = dbs['highlights'][idx]
  # Check if dt is a list before iterating
  if isinstance(dt, list):
    high = pd.DataFrame({
        'skuId' : dbs['skuId'][idx],
        'highlights' : [i['altText'] for i in dt if dt]
    })
    highlight_df.append(high)
  else:
    high = pd.DataFrame({
        'skuId' : [dbs['skuId'][idx]],
        'highlights' : [[]]
    })
    highlight_df.append(high)
highlight_df = pd.concat(highlight_df)
highlight_df = highlight_df.explode('highlights')
highlight_df.to_csv(f"{outputPath}/AgregateHighlights.csv",index=False)

## NLP on ShortDesc & Ingredients

### Ingredients

In [172]:
def getHighlightIng(text):
  text = text.replace("<br><br>Please be aware that ingredient lists may change or vary from time to time.  Please refer to the ingredient list on the product package you receive for the most up to date list of ingredients","")
  text = text.replace("<br><br>.<br><br><i>The list of ingredients is subject to change, please consult the packaging of the product purchased.</i>","")
  text = text.replace("<br><br><i>The list of ingredients is subject to change, please consult the packaging of the product purchased.</i>","")
  text = text.replace('<br><br>*','')
  if not '<br><br>' in text :
    StarIng = None
  else :
    StarIng = text.split('<br><br>')[0]
    StarIng = StarIng.split('<br>')
    StarIng = [{i.split(":")[0].replace('-','').lstrip() : i.split(":")[-1].lstrip().rstrip()}
               for i in StarIng]

  return StarIng

In [171]:
ingredientDesc = dbs[(dbs['Category3']!="Value & Gift Sets") & (dbs['Category2']!="Value & Gift Sets")  & (dbs['Category1']!="Value & Gift Sets") & (dbs['Category3']!="Eye Palettes")]
ingredientDesc = ingredientDesc[['skuId','ingredientDesc']]
ingredientDesc.drop_duplicates(subset="skuId",inplace=True)
ingredientDesc.dropna(inplace=True)
ingredientDesc.drop_duplicates(subset="skuId",inplace=True)
ingredientDesc.reset_index(drop=True,inplace=True)
ingredientDesc['starIng'] = ingredientDesc['ingredientDesc'].apply(lambda x : getHighlightIng(x))
ingredientDesc.dropna(inplace=True)
ingredientDesc = ingredientDesc.explode('starIng')
ingredientDesc.drop(columns=['ingredientDesc'],inplace=True)
ingredientDesc['Comments'] = ingrsedientDesc['starIng'].apply(lambda x : list(x.values())[0])
ingredientDesc['starIng'] = ingredientDesc['starIng'].apply(lambda x : list(x.keys())[0])
ingredientDesc.reset_index(drop=True,inplace=True)
ingredientDesc.to_csv(f"{outputPath}/starIngredients.csv",index=False)

### ShortDesc

In [186]:
shortDesc = dbs[(dbs['Category3']!="Value & Gift Sets") & (dbs['Category2']!="Value & Gift Sets")  & (dbs['Category1']!="Value & Gift Sets")]
shortDesc = shortDesc[['skuId','shortDesc']]
shortDesc.drop_duplicates(subset="skuId",inplace=True)

In [190]:
test = shortDesc["shortDesc"][0]